<a href="https://colab.research.google.com/github/AnaMoraes09/recomendacao_de_imoveis/blob/main/tratamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilizando o Spark no Google Colab

In [37]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [38]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [39]:
import findspark
findspark.init()

# Acessando Spark UI

In [40]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [41]:
spark

# Carregando dados

## Montando o drive

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Extraindo arquivo


In [43]:
import zipfile

In [44]:
zipfile.ZipFile('/content/drive/MyDrive/recomendacao_imoveis/semana-1.zip', 'r').extractall('/content/drive/MyDrive/recomendacao_imoveis')

In [45]:
path = '/content/drive/MyDrive/recomendacao_imoveis/dataset_bruto.json'
df = spark.read.json(path)

In [46]:
df.limit(5).toPandas()

,anuncio,imagens,usuario
0,"(0, [], [16], [0], [], (Centro, 20061003, Rio ...","[(39d6282a-71f3-47bc-94aa-909351ecd881, https:...","(9d44563d-3405-4e84-9381-35b7cf40a9a4, Frank)"
1,"(0, [], [14], [0], [], (Centro, 20051040, Rio ...","[(23d2b3ab-45b0-47f2-a27f-ffbe066c2c38, https:...","(36245be7-70fe-40cd-84d4-70a57dc8f976, Caroline)"
2,"(0, [1026], [1026], [0], [], (Maria da Graça, ...","[(1da65baa-368b-490f-bef7-1834b9a428c1, https:...","(9dc415d8-1397-4d8d-8425-b8310fad309d, Oliver)"
3,"(0, [120], [120], [0], [Portão eletrônico, Con...","[(79b542c6-49b4-4f06-80b5-ef4fcdf98f76, https:...","(9911a2df-f299-4a73-a384-281417abaf69, Matthew)"
4,"(0, [3], [3], [0], [], (São Cristóvão, 2093167...","[(e2bc497b-6510-4731-8942-97b3d3cdab5e, https:...","(240a7aab-12e5-4069-9a2c-9dca4c4f9c68, Jude)"


In [47]:
df.show(5,False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

## Importando bibliotecas

In [48]:
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql import functions as f

#  Tratamendo de dados

In [49]:
# Analisando o número de linhas e colunas 
print(f'temos {df.count()} linhas\
 e {len(df.columns)} colunas')

temos 89083 linhas e 3 colunas


In [50]:
# Avaliando a estrutura
df.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-



*   OBS: alguns itens da coluna 'anúncios' não estão no seu devido type



## Transformar cada campo da coluna "anuncio" em uma coluna separada

In [51]:
anuncio = df.select('anuncio.*')
anuncio.show(5,False)

+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                        |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|0    |[]    

## Filtrar a base de dados



*   O time de Data Science solicitou que fizéssemos alguns filtros nas colunas tipo_uso, tipo_unidade e tipo_anuncio da nossa base de dados:



In [52]:
anuncio\
  .select('*')\
  .filter(anuncio.tipo_uso.startswith('Residencial'))\
  .filter(anuncio.tipo_unidade.endswith('Apartamento'))\
  .filter(anuncio.tipo_anuncio.endswith('Usado'))\
  .limit(10)\
  .show(truncate = False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |endereco                                                                                                               |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                    |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+--------------

## Transformar as colunas dos cômodos dos imóveis de listas para inteiros

In [53]:
# Visualizando os tipos 
anuncio\
  .select("quartos", "suites", "banheiros", "vaga", "area_total" ,"area_util")\
  .printSchema()

root
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- vaga: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [54]:
# utilizei o '[0]' para mudar o primeiro e unico array
anuncio = anuncio\
  .withColumn("quartos", anuncio["quartos"][0].cast(IntegerType()))\
  .withColumn("suites", anuncio["suites"][0].cast(IntegerType()))\
  .withColumn("banheiros", anuncio["banheiros"][0].cast(IntegerType()))\
  .withColumn("vaga", anuncio["vaga"][0].cast(IntegerType()))\
  .withColumn("area_total", anuncio["area_total"][0].cast(IntegerType()))\
  .withColumn("area_util", anuncio["area_util"][0].cast(IntegerType()))

In [55]:
anuncio\
  .select("quartos", "suites", "banheiros", "vaga", "area_total" ,"area_util")\
  .printSchema()

root
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)



## Tratamento de informações sobre localização


In [56]:
# withColumn()é uma função de transformação do DataFrame que é usada para alterar
# o valor, converter o tipo de dados de uma coluna existente, criar uma nova coluna e muito mais. 

endereco = anuncio.select('endereco')
endereco.limit(5).show()

+--------------------+
|            endereco|
+--------------------+
|{Centro, 20061003...|
|{Centro, 20051040...|
|{Maria da Graça, ...|
|{Campo Grande, 23...|
|{São Cristóvão, 2...|
+--------------------+



In [57]:
anuncio = anuncio\
  .withColumn('bairro', endereco['endereco']['bairro'])\
  .withColumn('zona', endereco['endereco']['zona'])

anuncio.limit(5).show()


+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        bairro|        zona|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|        Centro|Zona Central|
|    0|      null|       14|        0|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260

## Transformar cada campo da coluna "valores" em uma coluna separada

In [58]:
valores = anuncio\
  .select('valores')
  
valores.limit(5).show(truncate = False)

+----------------------------+
|valores                     |
+----------------------------+
|[{260, 107, Venda, 10000}]  |
|[{260, 107, Venda, 10000}]  |
|[{null, 1613, Venda, 10000}]|
|[{80, null, Venda, 10000}]  |
|[{0, 0, Venda, 5000}]       |
+----------------------------+



In [59]:
anuncio = anuncio\
  .withColumn('condominio', valores[0]['condominio'])\
  .withColumn('iptu', valores[0]['iptu'])\
  .withColumn('tipo', valores[0]['tipo'])\
  .withColumn('valor', valores[0]['valor'])

anuncio.limit(5).show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+------+-------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        bairro|        zona|condominio|  iptu|   tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+------+-------+-------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|        Centro|Zona Central|     [260]| [107]|[Venda]|[10000]|
|    0|      null|       14|

## Tratamento para a coluna de valores

In [60]:
# Transformei o type da coluna 'tipo' de array para string 
anuncio = anuncio\
  .withColumn("tipo", anuncio["tipo"][0].cast(StringType()))

anuncio.limit(10).show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+------+-----+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        bairro|        zona|condominio|  iptu| tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+------+-----+-------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|        Centro|Zona Central|     [260]| [107]|Venda|[10000]|
|    0|      null|       14|        

In [61]:
anuncio_vendas = anuncio\
  .select('*')\
  .filter(anuncio.tipo.startswith('Venda'))

anuncio_vendas.limit(5).show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+------+-----+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        bairro|        zona|condominio|  iptu| tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+------+-----+-------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|        Centro|Zona Central|     [260]| [107]|Venda|[10000]|
|    0|      null|       14|        

## ultimas modificações antes de salvar 

In [62]:
# Excluindo colunas Endereço e valores 
anuncio_vendas = anuncio\
  .drop('endereco')\
  .drop('valores')
anuncio_vendas.limit(5).show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------+------------+----------+------+-----+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|        bairro|        zona|condominio|  iptu| tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------+------------+----------+------+-----+-------+
|    0|      null|       16|        0|                  []|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|        Centro|Zona Central|     [260]| [107]|Venda|[10000]|
|    0|      null|       14|        0|                  []|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|        Centro|Zona Central|     [260]| [107]|Venda|[10000]|
|    

In [64]:
anuncio_vendas = anuncio_vendas\
  .withColumn("condominio", anuncio_vendas["condominio"][0].cast(IntegerType()))\
  .withColumn("iptu", anuncio_vendas["iptu"][0].cast(IntegerType()))\
  .withColumn("valor", anuncio_vendas["valor"][0].cast(IntegerType()))

In [69]:
anuncio_vendas = anuncio_vendas\
  .withColumn("caracteristicas", anuncio_vendas["caracteristicas"][0].cast(StringType()))

anuncio_vendas.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: integer (nullable = true)
 |-- iptu: integer (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: integer (nullable = true)



## Salvando arquivo 

### Parquet

In [70]:
%%time
anuncio_vendas.write.parquet(
    path= '/content/drive/MyDrive/recomendacao_imoveis/dados',
    mode= 'overwrite')

CPU times: user 50.7 ms, sys: 4.96 ms, total: 55.7 ms
Wall time: 7.47 s


### CSV

In [71]:
%%time
anuncio_vendas.write.format("csv").save('/content/drive/MyDrive/recomendacao_imoveis/dados/csv')

CPU times: user 36.3 ms, sys: 2.54 ms, total: 38.8 ms
Wall time: 5.44 s
